In [77]:
# !pip3 install torch
# !pip install scikit-learn

In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [79]:
# get data
data = pd.read_csv("../usa/processed-usa-data.csv", delimiter=",", names=["Temperature_Bin", "Wind_Speed_Bin", "Amount_of_Accidents"], skiprows=1)
data = data.dropna()

In [86]:
from sklearn.model_selection import train_test_split

# Preprocess the data
data = data.dropna()

accidents = data["Amount_of_Accidents"].values
accidents = accidents.sum() / len(accidents)

print("average accidents: ", accidents)

# Perform one-hot encoding on the categorical columns
temperature_encoded = pd.get_dummies(data["Temperature_Bin"], prefix="temperature")
wind_speed_encoded = pd.get_dummies(data["Wind_Speed_Bin"], prefix="wind_speed")

# Combine the encoded features with the amount of accidents
processed_data = pd.concat([temperature_encoded, wind_speed_encoded, data["Amount_of_Accidents"]], axis=1)

# Convert the processed data to PyTorch tensors
X = processed_data.iloc[:, :-1].values
y = processed_data.iloc[:, -1].values

X_train, x_val, y_train,  y_val = train_test_split(X, y, test_size=0.2, shuffle=True)
 
X.shape
X_train.shape, x_val.shape

y_train
# Standardize the input features
# X_mean = torch.mean(X, dim=0)
# X_std = torch.std(X, dim=0)
# X_standardized = (X - X_mean) / X_std


# X.size(), y.size()

average accidents:  20240.4


array([    76,  15802,     45,  19331,  24186,     74,   9247,     28,
        43767,      5,  25962,    310,   1213,  10643,     75,  85158,
          116,   3273,  33567,    772,   8927,  34356, 136080,  69535,
       100558, 181655,  10728,    133,   2987, 111974, 162833,    223,
           47,   1484,     19,     24,   3536,    173,    507,  21627,
           35,  46526,     81,    534,   4713,  32693,  38569,   6050,
        10247,     29,   1915,    182,   9394,      7,     17,   7546,
        62352,   2018, 140630,    257,    414,   3503,  41564,     72,
          565,      0,   9824,   3048,    163,  27068,   2594,      0,
         5175,    146,  11877,  51595,  50086,      3,  19400,  56383])

In [81]:
# Standardize the input features
X_mean = torch.mean(X, dim=0)
X_std = torch.std(X, dim=0)
X_standardized = (X - X_mean) / X_std

TypeError: mean() received an invalid combination of arguments - got (numpy.ndarray, dim=int), but expected one of:
 * (Tensor input, *, torch.dtype dtype)
 * (Tensor input, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, tuple of names dim, bool keepdim, *, torch.dtype dtype, Tensor out)


In [ ]:
# Define the linear regression model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(X_standardized.shape[1], 1)  # Number of input features and 1 output

    def forward(self, x):
        return self.linear(x)

In [ ]:
# Initialize the model
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=3e-4)
# optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
# Train the model
num_epochs = 100000

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_standardized)
    loss = criterion(outputs, y)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 100/100000, Loss: 1358920960.0000
Epoch 200/100000, Loss: 1351550336.0000
Epoch 300/100000, Loss: 1351420928.0000
Epoch 400/100000, Loss: 1351418624.0000
Epoch 500/100000, Loss: 1351418624.0000
Epoch 600/100000, Loss: 1351418496.0000
Epoch 700/100000, Loss: 1351418624.0000
Epoch 800/100000, Loss: 1351418624.0000
Epoch 900/100000, Loss: 1351418624.0000
Epoch 1000/100000, Loss: 1351418624.0000
Epoch 1100/100000, Loss: 1351418624.0000
Epoch 1200/100000, Loss: 1351418624.0000
Epoch 1300/100000, Loss: 1351418624.0000
Epoch 1400/100000, Loss: 1351418624.0000
Epoch 1500/100000, Loss: 1351418624.0000
Epoch 1600/100000, Loss: 1351418624.0000
Epoch 1700/100000, Loss: 1351418624.0000
Epoch 1800/100000, Loss: 1351418624.0000
Epoch 1900/100000, Loss: 1351418624.0000
Epoch 2000/100000, Loss: 1351418624.0000
Epoch 2100/100000, Loss: 1351418624.0000
Epoch 2200/100000, Loss: 1351418624.0000
Epoch 2300/100000, Loss: 1351418624.0000
Epoch 2400/100000, Loss: 1351418624.0000
Epoch 2500/100000, Loss: 

In [101]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import numpy as np

lr = LinearRegression()
lr.fit(X_train, np.log(y_train + 1e-5))

rf = RandomForestRegressor()
rf.fit(X_train, np.log(y_train + 1e-5))

RandomForestRegressor()

In [102]:
lr.score(x_val, np.log(y_val + 1e-5)),rf.score(x_val, np.log(y_val + 1e-5))

(0.45312342691723884, 0.9372714312721295)

In [100]:
# Define the bins for temperature and wind speed
temperature_bins = [-float("inf"), -5, 0, 5, 10, 15, 20, 25, 30, 35, float("inf")]
wind_speed_bins = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, float("inf")]

# Assign labels to each bin
temperature_labels = [
    "Extremely Cold",
    "Very Cold",
    "Cold",
    "Slightly Cold",
    "Moderate Cold",
    "Moderate Hot",
    "Slightly Hot",
    "Hot",
    "Very Hot",
    "Extremely Hot",
]
wind_speed_labels = [
    "No Wind",
    "Very Low",
    "Low",
    "Slightly Low",
    "Moderate Low",
    "Moderate High",
    "Slightly High",
    "High",
    "Very High",
    "Extremely High",
]

current_temperature = 10
current_wind_speed = 10

# Encode the current temperature and wind speed values into one-hot encoding
temperature_index = next(i for i, bin in enumerate(temperature_bins) if current_temperature <= bin)
wind_speed_index = next(i for i, bin in enumerate(wind_speed_bins) if current_wind_speed <= bin)

encoded_temperature = [0] * len(temperature_labels)
encoded_temperature[temperature_index] = 1

encoded_wind_speed = [0] * len(wind_speed_labels)
encoded_wind_speed[wind_speed_index] = 1

# Combine the encoded values into a single input tensor
input_tensor = torch.tensor([encoded_temperature + encoded_wind_speed], dtype=torch.float32)

# Make the prediction using the trained model
predicted = model(input_tensor)

# Print the predicted amount of accidents
print(f"Predicted amount of accidents: {predicted.item():.2f}")
deviation = (predicted.item() - 20240.4) / 20240.4 * 100
print("percent deviation from average accidents per month: ", str(round(deviation, 3)) + "%")


Predicted amount of accidents: 20240.31
percent deviation from average accidents per month:  -0.0%
